In [1]:
import os
import json
import clang.cindex
import clang.enumerations
import csv
import numpy as np
import os


import re 

# set the config
clang.cindex.Config.set_library_path("/usr/lib/x86_64-linux-gnu")
clang.cindex.Config.set_library_file('/usr/lib/x86_64-linux-gnu/libclang-6.0.so.1')

In [3]:
# split_dir = '/home/saikatc/DATA/security-bias/VulnerabilityDetectionProject/Baseline/data/neurips_parsed/neurips_data/'
# parsed = '/home/saikatc/DATA/security-bias/VulnerabilityDetectionProject/Baseline/data/neurips_parsed/parsed_results/'
split_dir = '../data/neurips_parsed/neurips_data/'
parsed = '../data/neurips_parsed/parsed_results/'


files = os.listdir(split_dir)
print(len(files))
files = os.listdir(parsed)
print(len(files))
ggnn_json_data = json.load(open('/home/saikatc/DATA/CCS-Vul_Det/data/ggnn_input/' + \
                        'chrome_debian_cfg_full_text_files.json'))
files = [d['file_name'] for d in ggnn_json_data]
print(len(files))
print(files[0])


27318
27318
18513
5987_0.c


In [65]:
keywords = ["alignas", "alignof", "and", "and_eq", "asm", "atomic_cancel", "atomic_commit", 
            "atomic_noexcept", "auto", "bitand", "bitor", "bool", "break", "case", "catch", 
            "char", "char8_t", "char16_t", "char32_t", "class", "compl", "concept", "const", 
            "consteval", "constexpr", "constinit", "const_cast", "continue", "co_await", 
            "co_return", "co_yield", "decltype", "default", "delete", "do", "double", "dynamic_cast", 
            "else", "enum", "explicit", "export", "extern", "false", "float", "for", "friend", "goto", 
            "if", "inline", "int", "long", "mutable", "namespace", "new", "noexcept", "not", "not_eq", 
            "nullptr", "operator", "or", "or_eq", "private", "protected", "public", "reflexpr", 
            "register", "reinterpret_cast", "requires", "return", "short", "signed", "sizeof", "static", 
            "static_assert", "static_cast", "struct", "switch", "synchronized", "template", "this", 
            "thread_local", "throw", "true", "try", "typedef", "typeid", "typename", "union", "unsigned", 
            "using", "virtual", "void", "volatile", "wchar_t", "while", "xor", "xor_eq", "NULL"]
puncs = '~`!@#$%^&*()-+={[]}|\\;:\'\"<,>.?/'
puncs = list(puncs)

l_funcs = ['StrNCat', 'getaddrinfo', '_ui64toa', 'fclose', 'pthread_mutex_lock', 'gets_s', 'sleep', 
           '_ui64tot', 'freopen_s', '_ui64tow', 'send', 'lstrcat', 'HMAC_Update', '__fxstat', 'StrCatBuff', 
           '_mbscat', '_mbstok_s', '_cprintf_s', 'ldap_search_init_page', 'memmove_s', 'ctime_s', 'vswprintf', 
           'vswprintf_s', '_snwprintf', '_gmtime_s', '_tccpy', '*RC6*', '_mbslwr_s', 'random', 
           '__wcstof_internal', '_wcslwr_s', '_ctime32_s', 'wcsncat*', 'MD5_Init', '_ultoa', 
           'snprintf', 'memset', 'syslog', '_vsnprintf_s', 'HeapAlloc', 'pthread_mutex_destroy', 
           'ChangeWindowMessageFilter', '_ultot', 'crypt_r', '_strupr_s_l', 'LoadLibraryExA', '_strerror_s', 
           'LoadLibraryExW', 'wvsprintf', 'MoveFileEx', '_strdate_s', 'SHA1', 'sprintfW', 'StrCatNW', 
           '_scanf_s_l', 'pthread_attr_init', '_wtmpnam_s', 'snscanf', '_sprintf_s_l', 'dlopen', 
           'sprintfA', 'timed_mutex', 'OemToCharA', 'ldap_delete_ext', 'sethostid', 'popen', 'OemToCharW', 
           '_gettws', 'vfork', '_wcsnset_s_l', 'sendmsg', '_mbsncat', 'wvnsprintfA', 'HeapFree', '_wcserror_s', 
           'realloc', '_snprintf*', 'wcstok', '_strncat*', 'StrNCpy', '_wasctime_s', 'push*', '_lfind_s', 
           'CC_SHA512', 'ldap_compare_ext_s', 'wcscat_s', 'strdup', '_chsize_s', 'sprintf_s', 'CC_MD4_Init', 
           'wcsncpy', '_wfreopen_s', '_wcsupr_s', '_searchenv_s', 'ldap_modify_ext_s', '_wsplitpath', 
           'CC_SHA384_Final', 'MD2', 'RtlCopyMemory', 'lstrcatW', 'MD4', 'MD5', '_wcstok_s_l', '_vsnwprintf_s', 
           'ldap_modify_s', 'strerror', '_lsearch_s', '_mbsnbcat_s', '_wsplitpath_s', 'MD4_Update', '_mbccpy_s', 
           '_strncpy_s_l', '_snprintf_s', 'CC_SHA512_Init', 'fwscanf_s', '_snwprintf_s', 'CC_SHA1', 'swprintf', 
           'fprintf', 'EVP_DigestInit_ex', 'strlen', 'SHA1_Init', 'strncat', '_getws_s', 'CC_MD4_Final', 
           'wnsprintfW', 'lcong48', 'lrand48', 'write', 'HMAC_Init', '_wfopen_s', 'wmemchr', '_tmakepath', 
           'wnsprintfA', 'lstrcpynW', 'scanf_s', '_mbsncpy_s_l', '_localtime64_s', 'fstream.open', '_wmakepath', 
           'Connection.open', '_tccat', 'valloc', 'setgroups', 'unlink', 'fstream.put', 'wsprintfA', '*SHA1*', 
           '_wsearchenv_s', 'ualstrcpyA', 'CC_MD5_Update', 'strerror_s', 'HeapCreate', 'ualstrcpyW', '__xstat', 
           '_wmktemp_s', 'StrCatChainW', 'ldap_search_st', '_mbstowcs_s_l', 'ldap_modify_ext', '_mbsset_s', 
           'strncpy_s', 'move', 'execle', 'StrCat', 'xrealloc', 'wcsncpy_s', '_tcsncpy*', 'execlp', 
           'RIPEMD160_Final', 'ldap_search_s', 'EnterCriticalSection', '_wctomb_s_l', 'fwrite', '_gmtime64_s', 
           'sscanf_s', 'wcscat', '_strupr_s', 'wcrtomb_s', 'VirtualLock', 'ldap_add_ext_s', '_mbscpy', 
           '_localtime32_s', 'lstrcpy', '_wcsncpy*', 'CC_SHA1_Init', '_getts', '_wfopen', '__xstat64', 
           'strcoll', '_fwscanf_s_l', '_mbslwr_s_l', 'RegOpenKey', 'makepath', 'seed48', 'CC_SHA256', 
           'sendto', 'execv', 'CalculateDigest', 'memchr', '_mbscpy_s', '_strtime_s', 'ldap_search_ext_s', 
           '_chmod', 'flock', '__fxstat64', '_vsntprintf', 'CC_SHA256_Init', '_itoa_s', '__wcserror_s', 
           '_gcvt_s', 'fstream.write', 'sprintf', 'recursive_mutex', 'strrchr', 'gethostbyaddr', '_wcsupr_s_l', 
           'strcspn', 'MD5_Final', 'asprintf', '_wcstombs_s_l', '_tcstok', 'free', 'MD2_Final', 'asctime_s', 
           '_alloca', '_wputenv_s', '_wcsset_s', '_wcslwr_s_l', 'SHA1_Update', 'filebuf.sputc', 'filebuf.sputn', 
           'SQLConnect', 'ldap_compare', 'mbstowcs_s', 'HMAC_Final', 'pthread_condattr_init', '_ultow_s', 'rand', 
           'ofstream.put', 'CC_SHA224_Final', 'lstrcpynA', 'bcopy', 'system', 'CreateFile*', 'wcscpy_s', 
           '_mbsnbcpy*', 'open', '_vsnwprintf', 'strncpy', 'getopt_long', 'CC_SHA512_Final', '_vsprintf_s_l', 
           'scanf', 'mkdir', '_localtime_s', '_snprintf', '_mbccpy_s_l', 'memcmp', 'final', '_ultoa_s', 
           'lstrcpyW', 'LoadModule', '_swprintf_s_l', 'MD5_Update', '_mbsnset_s_l', '_wstrtime_s', '_strnset_s', 
           'lstrcpyA', '_mbsnbcpy_s', 'mlock', 'IsBadHugeWritePtr', 'copy', '_mbsnbcpy_s_l', 'wnsprintf', 
           'wcscpy', 'ShellExecute', 'CC_MD4', '_ultow', '_vsnwprintf_s_l', 'lstrcpyn', 'CC_SHA1_Final', 
           'vsnprintf', '_mbsnbset_s', '_i64tow', 'SHA256_Init', 'wvnsprintf', 'RegCreateKey', 'strtok_s', 
           '_wctime32_s', '_i64toa', 'CC_MD5_Final', 'wmemcpy', 'WinExec', 'CreateDirectory*', 
           'CC_SHA256_Update', '_vsnprintf_s_l', 'jrand48', 'wsprintf', 'ldap_rename_ext_s', 'filebuf.open', 
           '_wsystem', 'SHA256_Update', '_cwscanf_s', 'wsprintfW', '_sntscanf', '_splitpath', 'fscanf_s', 
           'strpbrk', 'wcstombs_s', 'wscanf', '_mbsnbcat_s_l', 'strcpynA', 'pthread_cond_init', 'wcsrtombs_s', 
           '_wsopen_s', 'CharToOemBuffA', 'RIPEMD160_Update', '_tscanf', 'HMAC', 'StrCCpy', 'Connection.connect', 
           'lstrcatn', '_mbstok', '_mbsncpy', 'CC_SHA384_Update', 'create_directories', 'pthread_mutex_unlock', 
           'CFile.Open', 'connect', '_vswprintf_s_l', '_snscanf_s_l', 'fputc', '_wscanf_s', '_snprintf_s_l', 
           'strtok', '_strtok_s_l', 'lstrcatA', 'snwscanf', 'pthread_mutex_init', 'fputs', 'CC_SHA384_Init', 
           '_putenv_s', 'CharToOemBuffW', 'pthread_mutex_trylock', '__wcstoul_internal', '_memccpy', 
           '_snwprintf_s_l', '_strncpy*', 'wmemset', 'MD4_Init', '*RC4*', 'strcpyW', '_ecvt_s', 'memcpy_s', 
           'erand48', 'IsBadHugeReadPtr', 'strcpyA', 'HeapReAlloc', 'memcpy', 'ldap_rename_ext', 'fopen_s', 
           'srandom', '_cgetws_s', '_makepath', 'SHA256_Final', 'remove', '_mbsupr_s', 'pthread_mutexattr_init', 
           '__wcstold_internal', 'StrCpy', 'ldap_delete', 'wmemmove_s', '_mkdir', 'strcat', '_cscanf_s_l', 
           'StrCAdd', 'swprintf_s', '_strnset_s_l', 'close', 'ldap_delete_ext_s', 'ldap_modrdn', 'strchr', 
           '_gmtime32_s', '_ftcscat', 'lstrcatnA', '_tcsncat', 'OemToChar', 'mutex', 'CharToOem', 'strcpy_s', 
           'lstrcatnW', '_wscanf_s_l', '__lxstat64', 'memalign', 'MD2_Init', 'StrCatBuffW', 'StrCpyN', 'CC_MD5', 
           'StrCpyA', 'StrCatBuffA', 'StrCpyW', 'tmpnam_r', '_vsnprintf', 'strcatA', 'StrCpyNW', '_mbsnbset_s_l', 
           'EVP_DigestInit', '_stscanf', 'CC_MD2', '_tcscat', 'StrCpyNA', 'xmalloc', '_tcslen', '*MD4*', 
           'vasprintf', 'strxfrm', 'chmod', 'ldap_add_ext', 'alloca', '_snscanf_s', 'IsBadWritePtr', 'swscanf_s', 
           'wmemcpy_s', '_itoa', '_ui64toa_s', 'EVP_DigestUpdate', '__wcstol_internal', '_itow', 'StrNCatW', 
           'strncat_s', 'ualstrcpy', 'execvp', '_mbccat', 'EVP_MD_CTX_init', 'assert', 'ofstream.write', 
           'ldap_add', '_sscanf_s_l', 'drand48', 'CharToOemW', 'swscanf', '_itow_s', 'RIPEMD160_Init', 
           'CopyMemory', 'initstate', 'getpwuid', 'vsprintf', '_fcvt_s', 'CharToOemA', 'setuid', 'malloc', 
           'StrCatNA', 'strcat_s', 'srand', 'getwd', '_controlfp_s', 'olestrcpy', '__wcstod_internal', 
           '_mbsnbcat', 'lstrncat', 'des_*', 'CC_SHA224_Init', 'set*', 'vsprintf_s', 'SHA1_Final', '_umask_s', 
           'gets', 'setstate', 'wvsprintfW', 'LoadLibraryEx', 'ofstream.open', 'calloc', '_mbstrlen', 
           '_cgets_s', '_sopen_s', 'IsBadStringPtr', 'wcsncat_s', 'add*', 'nrand48', 'create_directory', 
           'ldap_search_ext', '_i64toa_s', '_ltoa_s', '_cwscanf_s_l', 'wmemcmp', '__lxstat', 'lstrlen', 
           'pthread_condattr_destroy', '_ftcscpy', 'wcstok_s', '__xmknod', 'pthread_attr_destroy', 'sethostname', 
           '_fscanf_s_l', 'StrCatN', 'RegEnumKey', '_tcsncpy', 'strcatW', 'AfxLoadLibrary', 'setenv', 'tmpnam', 
           '_mbsncat_s_l', '_wstrdate_s', '_wctime64_s', '_i64tow_s', 'CC_MD4_Update', 'ldap_add_s', '_umask', 
           'CC_SHA1_Update', '_wcsset_s_l', '_mbsupr_s_l', 'strstr', '_tsplitpath', 'memmove', '_tcscpy', 
           'vsnprintf_s', 'strcmp', 'wvnsprintfW', 'tmpfile', 'ldap_modify', '_mbsncat*', 'mrand48', 'sizeof', 
           'StrCatA', '_ltow_s', '*desencrypt*', 'StrCatW', '_mbccpy', 'CC_MD2_Init', 'RIPEMD160', 'ldap_search', 
           'CC_SHA224', 'mbsrtowcs_s', 'update', 'ldap_delete_s', 'getnameinfo', '*RC5*', '_wcsncat_s_l', 
           'DriverManager.getConnection', 'socket', '_cscanf_s', 'ldap_modrdn_s', '_wopen', 'CC_SHA256_Final', 
           '_snwprintf*', 'MD2_Update', 'strcpy', '_strncat_s_l', 'CC_MD5_Init', 'mbscpy', 'wmemmove', 
           'LoadLibraryW', '_mbslen', '*alloc', '_mbsncat_s', 'LoadLibraryA', 'fopen', 'StrLen', 'delete', 
           '_splitpath_s', 'CreateFileTransacted*', 'MD4_Final', '_open', 'CC_SHA384', 'wcslen', 'wcsncat', 
           '_mktemp_s', 'pthread_mutexattr_destroy', '_snwscanf_s', '_strset_s', '_wcsncpy_s_l', 'CC_MD2_Final', 
           '_mbstok_s_l', 'wctomb_s', 'MySQL_Driver.connect', '_snwscanf_s_l', '*_des_*', 'LoadLibrary', 
           '_swscanf_s_l', 'ldap_compare_s', 'ldap_compare_ext', '_strlwr_s', 'GetEnvironmentVariable', 
           'cuserid', '_mbscat_s', 'strspn', '_mbsncpy_s', 'ldap_modrdn2', 'LeaveCriticalSection', 'CopyFile', 
           'getpwd', 'sscanf', 'creat', 'RegSetValue', 'ldap_modrdn2_s', 'CFile.Close', '*SHA_1*', 
           'pthread_cond_destroy', 'CC_SHA512_Update', '*RC2*', 'StrNCatA', '_mbsnbcpy', '_mbsnset_s', 
           'crypt', 'excel', '_vstprintf', 'xstrdup', 'wvsprintfA', 'getopt', 'mkstemp', '_wcsnset_s', 
           '_stprintf', '_sntprintf', 'tmpfile_s', 'OpenDocumentFile', '_mbsset_s_l', '_strset_s_l', 
           '_strlwr_s_l', 'ifstream.open', 'xcalloc', 'StrNCpyA', '_wctime_s', 'CC_SHA224_Update', '_ctime64_s', 
           'MoveFile', 'chown', 'StrNCpyW', 'IsBadReadPtr', '_ui64tow_s', 'IsBadCodePtr', 'getc', 
           'OracleCommand.ExecuteOracleScalar', 'AccessDataSource.Insert', 'IDbDataAdapter.FillSchema', 
           'IDbDataAdapter.Update', 'GetWindowText*', 'SendMessage', 'SqlCommand.ExecuteNonQuery', 'streambuf.sgetc', 
           'streambuf.sgetn', 'OracleCommand.ExecuteScalar', 'SqlDataSource.Update', '_Read_s', 'IDataAdapter.Fill', 
           '_wgetenv', '_RecordsetPtr.Open*', 'AccessDataSource.Delete', 'Recordset.Open*', 'filebuf.sbumpc', 'DDX_*', 
           'RegGetValue', 'fstream.read*', 'SqlCeCommand.ExecuteResultSet', 'SqlCommand.ExecuteXmlReader', 'main', 
           'streambuf.sputbackc', 'read', 'm_lpCmdLine', 'CRichEditCtrl.Get*', 'istream.putback', 
           'SqlCeCommand.ExecuteXmlReader', 'SqlCeCommand.BeginExecuteXmlReader', 'filebuf.sgetn', 
           'OdbcDataAdapter.Update', 'filebuf.sgetc', 'SQLPutData', 'recvfrom', 'OleDbDataAdapter.FillSchema', 
           'IDataAdapter.FillSchema', 'CRichEditCtrl.GetLine', 'DbDataAdapter.Update', 'SqlCommand.ExecuteReader', 
           'istream.get', 'ReceiveFrom', '_main', 'fgetc', 'DbDataAdapter.FillSchema', 'kbhit', 'UpdateCommand.Execute*', 
           'Statement.execute', 'fgets', 'SelectCommand.Execute*', 'getch', 'OdbcCommand.ExecuteNonQuery', 
           'CDaoQueryDef.Execute', 'fstream.getline', 'ifstream.getline', 'SqlDataAdapter.FillSchema', 
           'OleDbCommand.ExecuteReader', 'Statement.execute*', 'SqlCeCommand.BeginExecuteNonQuery', 
           'OdbcCommand.ExecuteScalar', 'SqlCeDataAdapter.Update', 'sendmessage', 'mysqlpp.DBDriver', 'fstream.peek', 
           'Receive', 'CDaoRecordset.Open', 'OdbcDataAdapter.FillSchema', '_wgetenv_s', 'OleDbDataAdapter.Update', 
           'readsome', 'SqlCommand.BeginExecuteXmlReader', 'recv', 'ifstream.peek', '_Main', '_tmain', '_Readsome_s', 
           'SqlCeCommand.ExecuteReader', 'OleDbCommand.ExecuteNonQuery', 'fstream.get', 'IDbCommand.ExecuteScalar', 
           'filebuf.sputbackc', 'IDataAdapter.Update', 'streambuf.sbumpc', 'InsertCommand.Execute*', 'RegQueryValue', 
           'IDbCommand.ExecuteReader', 'SqlPipe.ExecuteAndSend', 'Connection.Execute*', 'getdlgtext', 'ReceiveFromEx', 
           'SqlDataAdapter.Update', 'RegQueryValueEx', 'SQLExecute', 'pread', 'SqlCommand.BeginExecuteReader', 'AfxWinMain', 
           'getchar', 'istream.getline', 'SqlCeDataAdapter.Fill', 'OleDbDataReader.ExecuteReader', 'SqlDataSource.Insert', 
           'istream.peek', 'SendMessageCallback', 'ifstream.read*', 'SqlDataSource.Select', 'SqlCommand.ExecuteScalar', 
           'SqlDataAdapter.Fill', 'SqlCommand.BeginExecuteNonQuery', 'getche', 'SqlCeCommand.BeginExecuteReader', 'getenv', 
           'streambuf.snextc', 'Command.Execute*', '_CommandPtr.Execute*', 'SendNotifyMessage', 'OdbcDataAdapter.Fill', 
           'AccessDataSource.Update', 'fscanf', 'QSqlQuery.execBatch', 'DbDataAdapter.Fill', 'cin', 
           'DeleteCommand.Execute*', 'QSqlQuery.exec', 'PostMessage', 'ifstream.get', 'filebuf.snextc', 
           'IDbCommand.ExecuteNonQuery', 'Winmain', 'fread', 'getpass', 'GetDlgItemTextCCheckListBox.GetCheck', 
           'DISP_PROPERTY_EX', 'pread64', 'Socket.Receive*', 'SACommand.Execute*', 'SQLExecDirect', 
           'SqlCeDataAdapter.FillSchema', 'DISP_FUNCTION', 'OracleCommand.ExecuteNonQuery', 'CEdit.GetLine', 
           'OdbcCommand.ExecuteReader', 'CEdit.Get*', 'AccessDataSource.Select', 'OracleCommand.ExecuteReader', 
           'OCIStmtExecute', 'getenv_s', 'DB2Command.Execute*', 'OracleDataAdapter.FillSchema', 'OracleDataAdapter.Fill', 
           'CComboBox.Get*', 'SqlCeCommand.ExecuteNonQuery', 'OracleCommand.ExecuteOracleNonQuery', 'mysqlpp.Query', 
           'istream.read*', 'CListBox.GetText', 'SqlCeCommand.ExecuteScalar', 'ifstream.putback', 'readlink', 
           'CHtmlEditCtrl.GetDHtmlDocument', 'PostThreadMessage', 'CListCtrl.GetItemText', 'OracleDataAdapter.Update', 
           'OleDbCommand.ExecuteScalar', 'stdin', 'SqlDataSource.Delete', 'OleDbDataAdapter.Fill', 'fstream.putback', 
           'IDbDataAdapter.Fill', '_wspawnl', 'fwprintf', 'sem_wait', '_unlink', 'ldap_search_ext_sW', 'signal', 'PQclear', 
           'PQfinish', 'PQexec', 'PQresultStatus']

In [66]:
import os, sys, argparse
from graphviz import Digraph


def read_csv(csv_file_path):
    data = []
    with open(csv_file_path) as fp:
        header = fp.readline()
        header = header.strip()
        h_parts = [hp.strip() for hp in header.split('\t')]
        for line in fp:
            line = line.strip()
            instance = {}
            lparts = line.split('\t')
            for i, hp in enumerate(h_parts):
                if i < len(lparts):
                    content = lparts[i].strip()
                else:
                    content = ''
                instance[hp] = content
            data.append(instance)
        return data


def read_code_file(file_path):
    code_lines = {}
    with open(file_path) as fp:
        for ln, line in enumerate(fp):
            assert isinstance(line, str)
            line = line.strip()
            if '//' in line:
                line = line[:line.index('//')]
            code_lines[ln + 1] = line
        return code_lines


def extract_nodes_with_location_info(nodes):
    # Will return an array identifying the indices of those nodes in nodes array,
    # another array identifying the node_id of those nodes
    # another array indicating the line numbers
    # all 3 return arrays should have same length indicating 1-to-1 matching.
    node_indices = []
    node_ids = []
    line_numbers = []
    node_id_to_line_number = {}
    for node_index, node in enumerate(nodes):
        assert isinstance(node, dict)
        if 'location' in node.keys():
            location = node['location']
            if location == '':
                continue
            line_num = int(location.split(':')[0])
            node_id = node['key'].strip()
            node_indices.append(node_index)
            node_ids.append(node_id)
            line_numbers.append(line_num)
            node_id_to_line_number[node_id] = line_num
    return node_indices, node_ids, line_numbers, node_id_to_line_number
    pass


def create_adjacency_list(line_numbers, node_id_to_line_numbers, edges, data_dependency_only=False):
    adjacency_list = {}
    for ln in set(line_numbers):
        adjacency_list[ln] = [set(), set()]
    for edge in edges:
        edge_type = edge['type'].strip()
        if True :#edge_type in ['IS_AST_PARENT', 'FLOWS_TO']:
            start_node_id = edge['start'].strip()
            end_node_id = edge['end'].strip()
            if start_node_id not in node_id_to_line_numbers.keys() or end_node_id not in node_id_to_line_numbers.keys():
                continue
            start_ln = node_id_to_line_numbers[start_node_id]
            end_ln = node_id_to_line_numbers[end_node_id]
            if not data_dependency_only:
                if edge_type == 'CONTROLS': #Control Flow edges
                    adjacency_list[start_ln][0].add(end_ln)
            if edge_type == 'REACHES': # Data Flow edges
                adjacency_list[start_ln][1].add(end_ln)
    return adjacency_list


def create_visual_graph(code, adjacency_list, file_name='test_graph', verbose=False):
    graph = Digraph('Code Property Graph')
    for ln in adjacency_list:
        graph.node(str(ln), str(ln) + '\t' + code[ln], shape='box')
        control_dependency, data_dependency = adjacency_list[ln]
        for anode in control_dependency:
            graph.edge(str(ln), str(anode), color='red')
        for anode in data_dependency:
            graph.edge(str(ln), str(anode), color='blue')
    graph.render(file_name, view=verbose)


def create_forward_slice(adjacency_list, line_no):
    sliced_lines = set()
    sliced_lines.add(line_no)
    stack = list()
    stack.append(line_no)
    while len(stack) != 0:
        cur = stack.pop()
        if cur not in sliced_lines:
            sliced_lines.add(cur)
        adjacents = adjacency_list[cur]
        for node in adjacents:
            if node not in sliced_lines:
                stack.append(node)
    sliced_lines = sorted(sliced_lines)
    return sliced_lines


def combine_control_and_data_adjacents(adjacency_list):
    cgraph = {}
    for ln in adjacency_list:
        cgraph[ln] = set()
        cgraph[ln] = cgraph[ln].union(adjacency_list[ln][0])
        cgraph[ln] = cgraph[ln].union(adjacency_list[ln][1])
    return cgraph


def invert_graph(adjacency_list):
    igraph = {}
    for ln in adjacency_list.keys():
        igraph[ln] = set()
    for ln in adjacency_list:
        adj = adjacency_list[ln]
        for node in adj:
            igraph[node].add(ln)
    return igraph
    pass


def create_backward_slice(adjacency_list, line_no):
    inverted_adjacency_list = invert_graph(adjacency_list)
    return create_forward_slice(inverted_adjacency_list, line_no)

In [67]:
class Tokenizer:
    # creates the object, does the inital parse
    def __init__(self, path, tokenizer_type='original'):
        self.index = clang.cindex.Index.create()
        self.tu = self.index.parse(path)
        self.path = self.extract_path(path)
        self.symbol_table = {}
        self.symbol_count = 1
        self.tokenizer_type = tokenizer_type

    # To output for split_functions, must have same path up to last two folders
    def extract_path(self, path):
        return "".join(path.split("/")[:-2])

    
    def full_tokenize_cursor(self, cursor):
        tokens = cursor.get_tokens()
        result = []
        for token in tokens:
            if token.kind.name == "COMMENT":
                continue
            if token.kind.name == "LITERAL":
                result += self.process_literal(token)
                continue
            if token.kind.name == "IDENTIFIER":
                result += ["ID"]
                continue
            result += [token.spelling]
        return result

    def full_tokenize(self):
        cursor = self.tu.cursor
        return self.full_tokenize_cursor(cursor)

    def process_literal(self, literal):
        cursor_kind = clang.cindex.CursorKind
        kind = literal.cursor.kind
        if kind == cursor_kind.INTEGER_LITERAL:
            return literal.spelling
        if kind == cursor_kind.FLOATING_LITERAL:
            return literal.spelling
        if kind == cursor_kind.IMAGINARY_LITERAL:
            return ["NUM"]       
        if kind == cursor_kind.STRING_LITERAL:
            return ["STRING"]
        sp = literal.spelling
        if re.match('[0-9]+', sp) is not None:
            return sp
        return ["LITERAL"]

    def split_functions(self, method_only):
        results = []
        cursor_kind = clang.cindex.CursorKind
        cursor = self.tu.cursor
        for c in cursor.get_children():
            filename = c.location.file.name if c.location.file != None else "NONE"
            extracted_path = self.extract_path(filename)

            if (c.kind == cursor_kind.CXX_METHOD or (method_only == False and c.kind == cursor_kind.FUNCTION_DECL)) and extracted_path == self.path:
                name = c.spelling
                tokens = self.full_tokenize_cursor(c)
                filename = filename.split("/")[-1]
                results += [tokens]

        return results
    

def tokenize(file_text):
    try:
        c_file = open('/tmp/test1.c', 'w')
        c_file.write(file_text)
        c_file.close()
        tok = Tokenizer('/tmp/test1.c')
        results = tok.split_functions(False)
        return ' '.join(results[0])
    except:
        return None


In [68]:
def symbolic_tokenize(code):
    tokens = nltk.word_tokenize(code)
    c_tokens = []
    for t in tokens:
        if t.strip() != '':
            c_tokens.append(t.strip())
    f_count = 1
    var_count = 1
    symbol_table = {}
    final_tokens = []
    for idx in range(len(c_tokens)):
        t = c_tokens[idx]
        if t in keywords:
            final_tokens.append(t)
        elif t in puncs:
            final_tokens.append(t)
        elif t in l_funcs:
            final_tokens.append(t)
        elif idx < len(c_tokens) - 1 and c_tokens[idx + 1] == '(':
            if t in keywords:
                final_tokens.append(t)
            else:
                if t not in symbol_table.keys():
                    symbol_table[t] = "FUNC" + str(f_count)
                    f_count += 1
                final_tokens.append(symbol_table[t])
            idx += 1
        
        elif t.endswith('('):
            t = t[:-1]
            if t in keywords:
                final_tokens.append(t + '(')
            else:
                if t not in symbol_table.keys():
                    symbol_table[t] = "FUNC" + str(f_count)
                    f_count += 1
                final_tokens.append(symbol_table[t] + '(')
        elif t.endswith('()'):
            t = t[:-2]
            if t in keywords:
                final_tokens.append(t + '()')
            else:
                if t not in symbol_table.keys():
                    symbol_table[t] = "FUNC" + str(f_count)
                    f_count += 1
                final_tokens.append(symbol_table[t] + '()')
        elif re.match("^\"*\"$", t) is not None:
            final_tokens.append("STRING")
        elif re.match("^[0-9]+(\.[0-9]+)?$", t) is not None:
            final_tokens.append("NUMBER")
        elif re.match("^[0-9]*(\.[0-9]+)$", t) is not None:
            final_tokens.append("NUMBER")
        else:
            if t not in symbol_table.keys():
                symbol_table[t] = "VAR" + str(var_count)
                var_count += 1
            final_tokens.append(symbol_table[t])
    return ' '.join(final_tokens)


In [69]:
import csv 

def read_file(path):
    with open(path) as f:
        lines = f.readlines()
        return ' '.join(lines)
    
def extract_line_number(idx, nodes):
    while idx >= 0:
        c_node = nodes[idx]
        if 'location' in c_node.keys():
            location = c_node['location']
            if location.strip() != '':
                try:
                    ln = int(location.split(':')[0])
                    return ln
                except:
                    pass
        idx -= 1
    return -1


In [70]:
all_data = []
import nltk
from gensim.models import Word2Vec

ggnn_json_data = json.load(open('/home/saikatc/DATA/CCS-Vul_Det/data/ggnn_input/' + \
                        'devign_cfg_full_text_files.json'))
files = [d['file_name'] for d in ggnn_json_data]
print(len(files))

def reformat_code_line_graph(code_lines, adjacency_lists, lanel, wv_model_original, wv_model_li):
    actual_lines = []
    for ln in adjacency_lists.keys():
        cd, dd = adjacency_lists[ln]
        new_cd = [l for l in cd]
        new_dd = [l for l in dd] 
        actual_lines.extend(new_cd)
        actual_lines.extend(new_dd)
        actual_lines.append(ln)
    actual_lines = sorted(list(set(actual_lines)))
    line_no_to_idx = {}
    idx_to_line_no = {}
    for idx, ln in enumerate(actual_lines):
        line_no_to_idx[ln] = idx
        idx_to_line_no[idx] = ln
    data_point = {}
    graph = []
    for src in adjacency_lists.keys():
        cd, dd = adjacency_lists[src]
        for dest in cd:
            graph.append([line_no_to_idx[src], 0, line_no_to_idx[dest]])
            graph.append([line_no_to_idx[dest], 1, line_no_to_idx[src]])
        for dest in dd:
            graph.append([line_no_to_idx[src], 2, line_no_to_idx[dest]])
            graph.append([line_no_to_idx[dest], 3, line_no_to_idx[src]])
    original_tokens = []
    symbolic_tokens = []
    line_features_wv = []
    sym_line_features_wv = []
    
    for lidx in range(len(idx_to_line_no.keys())):
        actual_code_line = code_lines[idx_to_line_no[lidx]]
        actual_line_tokens = nltk.wordpunct_tokenize(actual_code_line)
        symbolic_line_tokens = symbolic_tokenize(actual_code_line).split()
        original_tokens.append(actual_line_tokens)
        symbolic_tokens.append(symbolic_line_tokens)
        
        nrp = np.zeros(100)
        for token in actual_line_tokens:
            try:
                embedding = wv_model_original.wv[token]
            except:
                embedding = np.zeros(100)
            nrp = np.add(nrp, embedding)
        if len(actual_line_tokens) > 0:
            fNrp = np.divide(nrp, len(actual_line_tokens))
        else:
            fNrp = nrp
        line_features_wv.append(fNrp.tolist())
        
        nrp = np.zeros(64)
        for token in symbolic_line_tokens:
            try:
                embedding = wv_model_li.wv[token]
            except:
                embedding = np.zeros(64)
            nrp = np.add(nrp, embedding)
        if len(actual_line_tokens) > 0:
            fNrp = np.divide(nrp, len(symbolic_line_tokens))
        else:
            fNrp = nrp
        sym_line_features_wv.append(fNrp.tolist())
    data_point = {
        'node_features': line_features_wv,
        'node_features_sym': sym_line_features_wv,
        'graph': graph,
        'original_tokens': original_tokens,
        'symbolic_tokens': symbolic_tokens,
        'targets': [[label]]
    }
    return data_point

26733


In [77]:
from tqdm import tqdm
wv_path = '../data/neurips_parsed/raw_code_neurips.100'
wv_model_original = Word2Vec.load(wv_path)
wv_model_li = Word2Vec.load('../data/Word2Vec/li_et_al_wv')
                                    
ggnn_data = []

for i, file_name  in enumerate(tqdm(files)):
    label = int(file_name.strip()[:-2].split('_')[-1])
    code_text = read_code_file(split_dir + file_name.strip())
    nodes_file_path = parsed + file_name.strip() + '/nodes.csv'
    edges_file_path = parsed + file_name.strip() + '/edges.csv'
    nc = open(nodes_file_path)
    nodes_file = csv.DictReader(nc, delimiter='\t')
    nodes = [node for node in nodes_file]
    if len(nodes) == 0:
        continue
    nodes = read_csv(nodes_file_path)
    edges = read_csv(edges_file_path)
    node_indices, node_ids, line_numbers, node_id_to_ln = extract_nodes_with_location_info(nodes)
    adjacency_list = create_adjacency_list(line_numbers, node_id_to_ln, edges, False)
    combined_graph = combine_control_and_data_adjacents(adjacency_list)
    data_point = reformat_code_line_graph(code_text, adjacency_list, label, wv_model_original, wv_model_li)
    ggnn_data.append(data_point)
    



  0%|          | 0/26733 [00:00<?, ?it/s]

  0%|          | 3/26733 [00:00<15:50, 28.12it/s]

  0%|          | 8/26733 [00:00<14:56, 29.80it/s]

  0%|          | 11/26733 [00:00<16:41, 26.68it/s]

  0%|          | 16/26733 [00:00<14:38, 30.40it/s]

  0%|          | 24/26733 [00:00<12:09, 36.59it/s]

  0%|          | 28/26733 [00:00<12:42, 35.04it/s]

  0%|          | 36/26733 [00:00<10:38, 41.81it/s]

  0%|          | 42/26733 [00:00<09:53, 44.99it/s]

  0%|          | 48/26733 [00:01<09:35, 46.33it/s]

  0%|          | 58/26733 [00:01<08:08, 54.58it/s]

  0%|          | 68/26733 [00:01<07:05, 62.74it/s]

  0%|          | 76/26733 [00:01<07:20, 60.58it/s]

  0%|          | 83/26733 [00:01<08:59, 49.43it/s]

  0%|          | 97/26733 [00:01<07:15, 61.18it/s]

  0%|          | 106/26733 [00:01<07:33, 58.75it/s]

  0%|          | 115/26733 [00:02<07:20, 60.47it/s]

  0%|          | 124/26733 [00:02<06:37, 66.92it/s]

  0%|          | 132/26733 [00:02<07:50, 56.55it/s]

  1%|          | 

  9%|▊         | 2290/26733 [00:36<09:46, 41.65it/s]

  9%|▊         | 2295/26733 [00:36<09:23, 43.39it/s]

  9%|▊         | 2301/26733 [00:37<08:37, 47.20it/s]

  9%|▊         | 2311/26733 [00:37<07:20, 55.49it/s]

  9%|▊         | 2318/26733 [00:37<07:25, 54.75it/s]

  9%|▊         | 2325/26733 [00:37<06:56, 58.57it/s]

  9%|▊         | 2333/26733 [00:37<06:56, 58.60it/s]

  9%|▉         | 2340/26733 [00:37<07:48, 52.05it/s]

  9%|▉         | 2348/26733 [00:37<07:10, 56.66it/s]

  9%|▉         | 2355/26733 [00:37<06:52, 59.13it/s]

  9%|▉         | 2362/26733 [00:37<06:33, 61.99it/s]

  9%|▉         | 2369/26733 [00:38<07:01, 57.81it/s]

  9%|▉         | 2376/26733 [00:38<07:47, 52.05it/s]

  9%|▉         | 2383/26733 [00:38<07:24, 54.79it/s]

  9%|▉         | 2391/26733 [00:38<06:45, 60.00it/s]

  9%|▉         | 2399/26733 [00:38<06:16, 64.58it/s]

  9%|▉         | 2411/26733 [00:38<05:42, 70.97it/s]

  9%|▉         | 2419/26733 [00:38<06:34, 61.67it/s]

  9%|▉         | 2426/26733 

 17%|█▋        | 4585/26733 [01:14<05:07, 71.95it/s]

 17%|█▋        | 4594/26733 [01:14<05:04, 72.82it/s]

 17%|█▋        | 4602/26733 [01:14<05:23, 68.32it/s]

 17%|█▋        | 4610/26733 [01:14<06:04, 60.69it/s]

 17%|█▋        | 4617/26733 [01:14<07:28, 49.36it/s]

 17%|█▋        | 4625/26733 [01:14<06:52, 53.62it/s]

 17%|█▋        | 4634/26733 [01:14<06:12, 59.38it/s]

 17%|█▋        | 4641/26733 [01:15<06:28, 56.84it/s]

 17%|█▋        | 4648/26733 [01:15<08:12, 44.87it/s]

 17%|█▋        | 4658/26733 [01:15<07:07, 51.59it/s]

 17%|█▋        | 4665/26733 [01:15<06:35, 55.82it/s]

 17%|█▋        | 4674/26733 [01:15<06:06, 60.14it/s]

 18%|█▊        | 4681/26733 [01:15<06:28, 56.72it/s]

 18%|█▊        | 4688/26733 [01:15<06:37, 55.42it/s]

 18%|█▊        | 4694/26733 [01:16<06:41, 54.92it/s]

 18%|█▊        | 4702/26733 [01:16<06:03, 60.54it/s]

 18%|█▊        | 4709/26733 [01:16<05:55, 62.03it/s]

 18%|█▊        | 4720/26733 [01:16<05:09, 71.15it/s]

 18%|█▊        | 4730/26733 

 26%|██▌       | 6937/26733 [01:52<05:16, 62.48it/s]

 26%|██▌       | 6946/26733 [01:52<04:56, 66.63it/s]

 26%|██▌       | 6957/26733 [01:52<04:23, 75.17it/s]

 26%|██▌       | 6966/26733 [01:52<04:17, 76.76it/s]

 26%|██▌       | 6980/26733 [01:52<03:43, 88.21it/s]

 26%|██▌       | 6991/26733 [01:53<03:31, 93.37it/s]

 26%|██▌       | 7004/26733 [01:53<03:19, 98.73it/s]

 26%|██▌       | 7017/26733 [01:53<03:06, 105.99it/s]

 26%|██▋       | 7031/26733 [01:53<02:56, 111.53it/s]

 26%|██▋       | 7044/26733 [01:53<02:52, 113.94it/s]

 26%|██▋       | 7056/26733 [01:53<03:05, 106.08it/s]

 26%|██▋       | 7067/26733 [01:53<03:18, 99.24it/s] 

 26%|██▋       | 7078/26733 [01:53<03:46, 86.62it/s]

 27%|██▋       | 7088/26733 [01:53<03:38, 89.85it/s]

 27%|██▋       | 7098/26733 [01:54<04:38, 70.61it/s]

 27%|██▋       | 7106/26733 [01:54<04:33, 71.82it/s]

 27%|██▋       | 7114/26733 [01:54<06:25, 50.93it/s]

 27%|██▋       | 7125/26733 [01:54<05:24, 60.38it/s]

 27%|██▋       | 7133/2

 36%|███▌      | 9525/26733 [02:29<05:21, 53.54it/s]

 36%|███▌      | 9532/26733 [02:29<05:44, 49.86it/s]

 36%|███▌      | 9541/26733 [02:30<05:17, 54.11it/s]

 36%|███▌      | 9554/26733 [02:30<04:28, 64.02it/s]

 36%|███▌      | 9563/26733 [02:30<04:11, 68.14it/s]

 36%|███▌      | 9571/26733 [02:30<04:09, 68.79it/s]

 36%|███▌      | 9581/26733 [02:30<03:55, 72.77it/s]

 36%|███▌      | 9590/26733 [02:30<03:57, 72.28it/s]

 36%|███▌      | 9598/26733 [02:30<04:00, 71.36it/s]

 36%|███▌      | 9606/26733 [02:30<03:56, 72.49it/s]

 36%|███▌      | 9614/26733 [02:31<04:11, 67.98it/s]

 36%|███▌      | 9622/26733 [02:31<04:19, 65.94it/s]

 36%|███▌      | 9636/26733 [02:31<03:42, 76.92it/s]

 36%|███▌      | 9649/26733 [02:31<03:21, 84.99it/s]

 36%|███▌      | 9659/26733 [02:31<03:22, 84.19it/s]

 36%|███▌      | 9668/26733 [02:31<03:36, 78.87it/s]

 36%|███▌      | 9680/26733 [02:31<03:21, 84.65it/s]

 36%|███▌      | 9689/26733 [02:31<03:26, 82.69it/s]

 36%|███▋      | 9698/26733 

...

 54%|█████▍    | 14422/26733 [03:44<02:49, 72.69it/s]

 54%|█████▍    | 14430/26733 [03:44<03:46, 54.24it/s]

 54%|█████▍    | 14437/26733 [03:45<03:40, 55.86it/s]

 54%|█████▍    | 14449/26733 [03:45<03:08, 65.30it/s]

 54%|█████▍    | 14457/26733 [03:45<03:27, 59.21it/s]

 54%|█████▍    | 14470/26733 [03:45<02:53, 70.62it/s]

 54%|█████▍    | 14479/26733 [03:45<02:45, 73.89it/s]

 54%|█████▍    | 14488/26733 [03:45<02:46, 73.40it/s]

 54%|█████▍    | 14497/26733 [03:45<03:00, 67.71it/s]

 54%|█████▍    | 14511/26733 [03:45<02:36, 78.18it/s]

 54%|█████▍    | 14523/26733 [03:46<02:20, 86.62it/s]

 54%|█████▍    | 14533/26733 [03:46<02:35, 78.27it/s]

 54%|█████▍    | 14542/26733 [03:46<02:53, 70.06it/s]

 54%|█████▍    | 14551/26733 [03:46<02:48, 72.46it/s]

 54%|█████▍    | 14562/26733 [03:46<02:34, 78.97it/s]

 55%|█████▍    | 14571/26733 [03:46<02:37, 77.38it/s]

 55%|█████▍    | 14580/26733 [03:46<02:34, 78.46it/s]

 55%|█████▍    | 14589/26733 [03:46<02:40, 75.46it/s]

 55%|█████

 64%|██████▍   | 17073/26733 [04:21<03:02, 52.80it/s]

 64%|██████▍   | 17081/26733 [04:22<03:04, 52.18it/s]

 64%|██████▍   | 17094/26733 [04:22<02:56, 54.76it/s]

 64%|██████▍   | 17103/26733 [04:22<02:45, 58.04it/s]

 64%|██████▍   | 17110/26733 [04:22<03:07, 51.36it/s]

 64%|██████▍   | 17116/26733 [04:22<03:22, 47.58it/s]

 64%|██████▍   | 17122/26733 [04:22<03:20, 47.93it/s]

 64%|██████▍   | 17129/26733 [04:23<03:16, 48.75it/s]

 64%|██████▍   | 17137/26733 [04:23<02:54, 55.00it/s]

 64%|██████▍   | 17151/26733 [04:23<02:23, 66.62it/s]

 64%|██████▍   | 17160/26733 [04:23<03:10, 50.20it/s]

 64%|██████▍   | 17170/26733 [04:23<02:43, 58.40it/s]

 64%|██████▍   | 17178/26733 [04:23<02:40, 59.51it/s]

 64%|██████▍   | 17186/26733 [04:23<02:28, 64.43it/s]

 64%|██████▍   | 17194/26733 [04:23<02:25, 65.56it/s]

 64%|██████▍   | 17202/26733 [04:24<02:47, 57.03it/s]

 64%|██████▍   | 17209/26733 [04:24<02:50, 55.75it/s]

 64%|██████▍   | 17220/26733 [04:24<02:36, 60.82it/s]

 64%|█████

 73%|███████▎  | 19400/26733 [05:00<01:42, 71.85it/s]

 73%|███████▎  | 19408/26733 [05:00<02:12, 55.47it/s]

 73%|███████▎  | 19415/26733 [05:00<02:04, 58.73it/s]

 73%|███████▎  | 19424/26733 [05:01<01:53, 64.51it/s]

 73%|███████▎  | 19432/26733 [05:01<01:51, 65.29it/s]

 73%|███████▎  | 19441/26733 [05:01<01:50, 66.04it/s]

 73%|███████▎  | 19448/26733 [05:01<01:54, 63.54it/s]

 73%|███████▎  | 19455/26733 [05:01<02:04, 58.36it/s]

 73%|███████▎  | 19463/26733 [05:01<02:02, 59.52it/s]

 73%|███████▎  | 19474/26733 [05:01<01:45, 68.92it/s]

 73%|███████▎  | 19482/26733 [05:01<01:55, 63.05it/s]

 73%|███████▎  | 19489/26733 [05:02<01:51, 64.80it/s]

 73%|███████▎  | 19496/26733 [05:02<03:11, 37.71it/s]

 73%|███████▎  | 19504/26733 [05:02<02:43, 44.12it/s]

 73%|███████▎  | 19510/26733 [05:02<02:42, 44.32it/s]

 73%|███████▎  | 19516/26733 [05:02<02:39, 45.17it/s]

 73%|███████▎  | 19523/26733 [05:02<02:25, 49.50it/s]

 73%|███████▎  | 19535/26733 [05:02<02:01, 59.30it/s]

 73%|█████

 82%|████████▏ | 21974/26733 [05:36<01:07, 70.20it/s]

 82%|████████▏ | 21986/26733 [05:36<00:59, 79.64it/s]

 82%|████████▏ | 21997/26733 [05:36<00:56, 83.61it/s]

 82%|████████▏ | 22006/26733 [05:37<01:01, 76.52it/s]

 82%|████████▏ | 22015/26733 [05:37<01:12, 65.22it/s]

 82%|████████▏ | 22023/26733 [05:37<01:28, 53.16it/s]

 82%|████████▏ | 22036/26733 [05:37<01:14, 63.33it/s]

 82%|████████▏ | 22048/26733 [05:37<01:05, 71.01it/s]

 83%|████████▎ | 22065/26733 [05:37<00:54, 85.28it/s]

 83%|████████▎ | 22078/26733 [05:37<00:50, 93.07it/s]

 83%|████████▎ | 22090/26733 [05:38<00:53, 87.00it/s]

 83%|████████▎ | 22100/26733 [05:38<01:02, 74.44it/s]

 83%|████████▎ | 22110/26733 [05:38<00:58, 78.51it/s]

 83%|████████▎ | 22119/26733 [05:38<00:59, 76.91it/s]

 83%|████████▎ | 22129/26733 [05:38<00:56, 82.21it/s]

 83%|████████▎ | 22142/26733 [05:38<00:50, 90.84it/s]

 83%|████████▎ | 22152/26733 [05:38<00:55, 82.38it/s]

 83%|████████▎ | 22164/26733 [05:38<00:50, 90.43it/s]

 83%|█████

 92%|█████████▏| 24568/26733 [06:14<00:31, 67.71it/s]

 92%|█████████▏| 24576/26733 [06:14<00:32, 67.33it/s]

 92%|█████████▏| 24584/26733 [06:14<00:32, 66.75it/s]

 92%|█████████▏| 24591/26733 [06:14<00:35, 60.15it/s]

 92%|█████████▏| 24598/26733 [06:15<00:43, 49.10it/s]

 92%|█████████▏| 24604/26733 [06:15<00:41, 51.45it/s]

 92%|█████████▏| 24610/26733 [06:15<00:40, 52.19it/s]

 92%|█████████▏| 24617/26733 [06:15<00:39, 53.54it/s]

 92%|█████████▏| 24624/26733 [06:15<00:37, 56.98it/s]

 92%|█████████▏| 24630/26733 [06:15<00:39, 52.76it/s]

 92%|█████████▏| 24642/26733 [06:15<00:33, 62.55it/s]

 92%|█████████▏| 24652/26733 [06:15<00:29, 69.51it/s]

 92%|█████████▏| 24660/26733 [06:15<00:30, 68.60it/s]

 92%|█████████▏| 24673/26733 [06:16<00:25, 79.74it/s]

 92%|█████████▏| 24683/26733 [06:16<00:30, 67.33it/s]

 92%|█████████▏| 24693/26733 [06:16<00:27, 73.67it/s]

 92%|█████████▏| 24702/26733 [06:16<00:29, 67.79it/s]

 92%|█████████▏| 24719/26733 [06:16<00:24, 81.84it/s]

 93%|█████

In [78]:
len(ggnn_data)

26733

In [79]:
fp = open('../data/line_ggnn/devign-line_ggnn.json', 'w')
json.dump(ggnn_data, fp)
fp.close()